In [1]:
!pip install datasets transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


In [2]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np

# Load pre-trained XLM-R model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Load the dataset
dataset = load_dataset("amazon_polarity")

# Function to select a percentage of the data
def select_subset(dataset, percentage):
    num_samples = len(dataset)
    num_to_select = int(percentage * num_samples)
    indices = np.random.choice(num_samples, size=num_to_select, replace=False)
    return dataset.select(indices)

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples['content'], truncation=True, padding='max_length', max_length=128)

# Select a subset of the data (e.g., 5%)
train_subset = select_subset(dataset['train'], 0.005)
test_subset = select_subset(dataset['test'], 0.005)

# Apply tokenization to the subsets
encoded_train_dataset = train_subset.map(preprocess_function, batched=True)
encoded_test_dataset = test_subset.map(preprocess_function, batched=True)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./xlm-r_finetuned_sentiment")
tokenizer.save_pretrained("./xlm-r_finetuned_sentiment")

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.217100,0.246122
2,0.194000,0.270858
3,0.095700,0.366061


Evaluation results: {'eval_loss': 0.36606109142303467, 'eval_runtime': 13.7173, 'eval_samples_per_second': 145.801, 'eval_steps_per_second': 9.113, 'epoch': 3.0}


In [3]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
import torch

# Load the fine-tuned model and tokenizer
model_path = "./xlm-r_finetuned_sentiment"
model = XLMRobertaForSequenceClassification.from_pretrained(model_path)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Generate prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()

    # Map the predicted class to sentiment
    sentiment = "Positive" if predicted_class == 1 else "Negative"
    confidence = probabilities[0][predicted_class].item()

    return sentiment, confidence

# Example usage
example_texts = [
    "This product exceeded my expectations. It's fantastic!",
    "I'm very disappointed with the quality. Would not recommend.",
    "It's okay, but not great. There's room for improvement.",
]

for text in example_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"Predicted sentiment: {sentiment}")
    print(f"Confidence: {confidence:.2f}")
    print()

# Interactive mode
print("Enter your own text for sentiment analysis (type 'quit' to exit):")
while True:
    user_input = input("Text: ")
    if user_input.lower() == 'quit':
        break
    sentiment, confidence = predict_sentiment(user_input)
    print(f"Predicted sentiment: {sentiment}")
    print(f"Confidence: {confidence:.2f}")
    print()

print("Thank you for using the sentiment analysis tool!")

Text: This product exceeded my expectations. It's fantastic!
Predicted sentiment: Positive
Confidence: 1.00

Text: I'm very disappointed with the quality. Would not recommend.
Predicted sentiment: Negative
Confidence: 1.00

Text: It's okay, but not great. There's room for improvement.
Predicted sentiment: Negative
Confidence: 0.99

Enter your own text for sentiment analysis (type 'quit' to exit):
Text: Thats some fine data
Predicted sentiment: Positive
Confidence: 1.00

Text: quit
Thank you for using the sentiment analysis tool!


In [4]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Evaluation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    macro_f1 = f1_score(labels, preds, average='macro')
    micro_f1 = f1_score(labels, preds, average='micro')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'macro_f1': macro_f1,
        'micro_f1': micro_f1
    }

# Run evaluation
trainer.evaluate()


{'eval_loss': 0.36606109142303467,
 'eval_runtime': 14.6349,
 'eval_samples_per_second': 136.66,
 'eval_steps_per_second': 8.541,
 'epoch': 3.0}

In [6]:
import torch
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load model and tokenizer
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base")
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

@app.route('/predict', methods=['POST'])
def predict():
    text = request.json['text']
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        output = model(**tokens)

    pred = torch.argmax(output.logits, dim=1).item()
    sentiment = 'positive' if pred == 1 else 'negative'

    return jsonify({'sentiment': sentiment})

# Start the server
if __name__ == '__main__':
    app.run(debug=True)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [7]:
@app.route('/feedback', methods=['POST'])
def feedback():
    text = request.json['text']
    correct_sentiment = request.json['correct_sentiment']

    # Log feedback for future retraining
    with open('feedback_log.csv', 'a') as f:
        f.write(f"{text},{correct_sentiment}\n")

    return jsonify({'message': 'Feedback recorded. Thanks!'})

# Use feedback in future training
def retrain_with_feedback():
    feedback_data = pd.read_csv('feedback_log.csv')
    # Add feedback data to the training dataset and retrain
    # trainer.train(feedback_dataset)
